# Data Pipeline Template  
A repeatable pipeline for building analysis-ready datasets with validation.

## Dataset Overview  

**Describe the dataset here:**
* What does a row represent?
* What are the key variables?
* What makes this data messy/realistic?
* What is the time range?

## What this Pipeline Produces

**Artifacts**
* `data/raw/` - raw data snapshots + metadata
* `data/staged/` - parsed/normalized table (typed, missingness normalized)
* `data/warehouse/` - curated table (Parquet; optionally partitioned)
* `data/reference/validation_report.json` - contracts + anomaly rates + canaries
* `data/reference/pipeline_runs/` - run logs for reproducibility

# 0. Setup

## 0.1 Directory Structure

Create project directories for the pipeline layers

In [4]:
# Project structure setup

# Import Libraries
from __future__ import annotations

from pathlib import Path
from datetime import datetime, timedelta, timezone
import json
import hashlib
import math

import numpy as np
import pandas as pd

from IPython.display import display

pd.set_option("display.max_columns", 180)
pd.set_option("display.width", 180)

# Define Paths
WORK_DIR = Path("work")
PROJECT_DIR = WORK_DIR / "wadi_A1"
DATA_DIR = PROJECT_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
STAGED_DIR = DATA_DIR / "staged"
WH_DIR = DATA_DIR / "warehouse"
REF_DIR = DATA_DIR / "reference"
RUN_DIR = REF_DIR / "pipeline_runs"

# Create Directories
for p in [RAW_DIR, STAGED_DIR, WH_DIR, REF_DIR, RUN_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# Output
print("Project:", PROJECT_DIR)
print("Raw:", RAW_DIR)
print("Staged:", STAGED_DIR)
print("Warehouse:", WH_DIR)
print("Reference:", REF_DIR)
print("Runs:", RUN_DIR)


Project: work/wadi_A1
Raw: work/wadi_A1/data/raw
Staged: work/wadi_A1/data/staged
Warehouse: work/wadi_A1/data/warehouse
Reference: work/wadi_A1/data/reference
Runs: work/wadi_A1/data/reference/pipeline_runs


## 0.2 Helper Utilities

Define reusable helper functions for the pipeline

In [5]:
# Helper Functions
class PipelineError(RuntimeError):
    pass

def utc_now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def sha16(x: str) -> str:
    return hashlib.sha256(x.encode('utf-8')).hexdigest()[:16]

def write_json(path: Path, obj: dict) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2, default=str))

def read_json(path: Path) -> dict:
    return json.loads(path.read_text())

def require_columns(df: pd.DataFrame, cols: list[str], context: str) -> None:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise PipelineError(f'[{context}] Missing required columns: {missing}')

def require_unique(df: pd.DataFrame, key: str, context: str) -> None:
    if key not in df.columns:
        raise PipelineError(f'[{context}] Missing key column "{key}"')
    dupes = int(df[key].duplicated().sum())
    if dupes:
        raise PipelineError(f'[{context}] Key "{key}" has {dupes} duplicates')

print("Helpers ready.")


Helpers ready.


## 0.3 Configuration

Set pipeline configuration constants

In [6]:
# Pipeline Configuration

# Dataset identifiers
DATA_SOURCE = "iTrust Labs"
DATA_URL = "https://itrust.sutd.edu.sg/itrust-labs_overview/"
STATION_ID = "Water Distribution (WaDi)"
STATION_URL = "https://itrust.sutd.edu.sg/itrust-labs-home/itrust-labs_wadi/"
DATASET_NAME = "WaDi.A1_9 Oct 2017"
DATASET_URL = "https://itrust.sutd.edu.sg/itrust-labs_datasets/dataset_info/"

# Run identification
RUN_ID = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S_UTC")

# Any other constants

---

# 1. Ingest: Acquire Raw Data

This run will use the iTrust Labs WaDi project testbed.  
Dataset **WaDi.A1**

The dataset can not be automatically downloaded. The following needs to be accomplished:
* Request dataset and fill out registration and agreement
* Receive download link to .zip file
* Unzip file for access to the .csv file
* https://itrust.sutd.edu.sg/itrust-labs_datasets/dataset_info/

## 1.1 Fetch Raw Data

**Dataset was downloaded 02/06/2026**  
**WaDi.A1_9 Oct 2017**

**The following files are included with the dataset:**
* table_WADI.pdf
* attack_description.xlsx
* WADI_14days.csv
* WADI_attackdata.csv

The files have been saved in location:  
`work/wadi_A1/data/raw/`

In [7]:
# Fetch Dataset

# Define expected files
EXPECTED_FILES = [
    'WADI_14days.csv',
    'WADI_attackdata.csv', 
    'attack_description.xlsx',
    'table_WADI.pdf'
]

# Verify all files exist
missing_files = [f for f in EXPECTED_FILES if not (RAW_DIR / f).exists()]

if missing_files:
    raise PipelineError(f"Missing required files in {RAW_DIR}: {missing_files}")

print("All required files present in RAW_DIR:")
for filename in EXPECTED_FILES:
    print(f" {filename}")



All required files present in RAW_DIR:
 WADI_14days.csv
 WADI_attackdata.csv
 attack_description.xlsx
 table_WADI.pdf


## 1.2 Write Raw Metadata

Document the raw data source and retrieval details

In [8]:
# Create Raw Metadata
file_metadata = {}
for filename in EXPECTED_FILES:
    file_path = RAW_DIR / filename
    file_metadata[filename] = {
        'size_bytes': file_path.stat().st_size,
        'path': str(file_path)
    }

metadata_raw = {
    'run_id': RUN_ID,
    'fetched_at_utc': utc_now_iso(),
    'source': DATA_SOURCE,
    'source_url': DATASET_URL,
    'dataset_name': DATASET_NAME,
    'download_date': '2026-02-06',
    'files': file_metadata,
    'notes': [
        'Dataset requires registration with iTrust Labs',
        'Files manually downloaded and placed in RAW_DIR',
        'Contains normal operations (14 days) and attack scenarios'
    ]
}

# Save metadata
metadata_path_raw = RAW_DIR / f"metadata_raw_{RUN_ID}.json"
write_json(metadata_path_raw, metadata_raw)
print(f"Raw metadata saved: {metadata_path_raw}")

Raw metadata saved: work/wadi_A1/data/raw/metadata_raw_20260207_142130_UTC.json


---

# 2. Stage: Parse and Normalize

Convert raw data into a clean, typed DataFrame

## 2.1 Read Raw Data

**How do we read it?**

In [28]:
# Read raw data
df_raw_normal = pd.read_csv(RAW_DIR / 'WADI_14days.csv', skiprows=4)
df_raw_attack = pd.read_csv(RAW_DIR / 'WADI_attackdata.csv')
df_raw_attack_meta = pd.read_excel(RAW_DIR / 'attack_description.xlsx', header=4)
df_raw_attack_meta = df_raw_attack_meta.dropna(subset=['S.No'])

print(f"Normal ops: {df_raw_normal.shape}")
print(f"Attack data: {df_raw_attack.shape}")
print(f"Attack metadata: {df_raw_attack_meta.shape}")

print("RAW Normal Data")
display(df_raw_normal.head(3))

print("RAW Attack Data")
display(df_raw_attack.head(3))

Normal ops: (1209601, 130)
Attack data: (172801, 130)
Attack metadata: (15, 9)
RAW Normal Data


,Row,Date,Time,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_004_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_005_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_001_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_002_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_005_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_006_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_DPIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_101_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_101_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_101_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_201_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_201_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_201_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_301_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_301_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_301_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_401_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_401_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_401_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_501_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_501_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_501_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_601_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_601_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_601_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_101_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_201_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_301_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_401_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_501_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_601_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_001_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_002_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_101_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_101_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_201_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_201_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_301_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_301_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_401_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_401_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_501_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_501_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_601_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_601_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_007_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_101_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_201_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_301_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_

RAW Attack Data


,Row,Date,Time,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_004_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_005_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_001_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_002_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_005_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_P_006_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_DPIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_101_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_101_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_101_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_201_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_201_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_201_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_301_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_301_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_301_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_401_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_401_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_401_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_501_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_501_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_501_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_601_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_601_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIC_601_SP,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_101_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_201_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_301_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_401_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_501_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_FQ_601_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_001_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_002_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_101_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_101_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_201_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_201_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_301_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_301_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_401_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_401_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_501_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_501_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_601_AH,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LS_601_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_LT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_007_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_101_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_201_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\2_MCV_301_CO,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_

## 2.2 Create Staged DataFrame

**How do we transform it?**

In [29]:
# Create staged DataFrame

# >>> TODO: Define stage_data(df_raw) function
# Create ID column
# Parse timestamps (timezone-aware UTC)
# Normalize missing values -> NaN
# Coerce to proper types
# Sort by timestamp
# Deduplicate
# Return Staged DataFrame

# Inspect columns
print(f"Total columns: {len(df_raw_normal.columns)}")
print(f"\nFirst 10 columns:")
print(df_raw_normal.columns[:10].tolist())
print(f"\nColumn types:")
print(df_raw_normal.dtypes.head(10))

def stage_wadi_data(df_raw: pd.DataFrame, dataset_id: str) -> pd.Dataframe:
    """
    Strip path prefix, leaving just sensor names
    Parse timestamp
    Drop row column
    Add dataset identifier
    """

    df = df_raw.copy()

    # Clean column names
    rename_map = {}
    for col in df.columns:
        if col.startswith('\\\\WIN'):
            sensor_name = col.split('\\')[-1]
            rename_map[col] = sensor_name

    df = df.rename(columns=rename_map)

    # Parse timestamp
    # df['time_utc'] = pd.to_datetime(df['Date'] + ' ' + df['Time']).dt.tz_localize('UTC')

    # Parse timestamp - let pandas auto-detect format
    datetime_str = df['Date'] + ' ' + df['Time']
    df['time_utc'] = pd.to_datetime(datetime_str, format='mixed').dt.tz_localize('UTC')
    
    # Add dataset identifier
    df['dataset_id'] = dataset_id

    # Drop metadata columns
    df = df.drop(columns=['Row', 'Date', 'Time'])

    # Reorder: dataset_id, time_utc, then all sensors
    sensor_cols = [col for col in df.columns if col not in ['dataset_id', 'time_utc']]
    df = df[['dataset_id', 'time_utc'] + sensor_cols]

    # Sort by time
    df = df.sort_values('time_utc').reset_index(drop=True)

    return df

print ("Staging function ready")

Total columns: 130

First 10 columns:
['Row', 'Date', 'Time', '\\\\WIN-25J4RO10SBF\\LOG_DATA\\SUTD_WADI\\LOG_DATA\\1_AIT_001_PV', '\\\\WIN-25J4RO10SBF\\LOG_DATA\\SUTD_WADI\\LOG_DATA\\1_AIT_002_PV', '\\\\WIN-25J4RO10SBF\\LOG_DATA\\SUTD_WADI\\LOG_DATA\\1_AIT_003_PV', '\\\\WIN-25J4RO10SBF\\LOG_DATA\\SUTD_WADI\\LOG_DATA\\1_AIT_004_PV', '\\\\WIN-25J4RO10SBF\\LOG_DATA\\SUTD_WADI\\LOG_DATA\\1_AIT_005_PV', '\\\\WIN-25J4RO10SBF\\LOG_DATA\\SUTD_WADI\\LOG_DATA\\1_FIT_001_PV', '\\\\WIN-25J4RO10SBF\\LOG_DATA\\SUTD_WADI\\LOG_DATA\\1_LS_001_AL']

Column types:
Row                                                             int64
Date                                                           object
Time                                                           object
\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_001_PV    float64
\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_002_PV    float64
\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_003_PV    float64
\\WIN-25J4RO10SBF\LOG_DATA\S

## 2.3 Test Staging Function

**Does it work?**

In [30]:
# Test staged function

# Stage normal operations data
df_staged_normal = stage_wadi_data(df_raw_normal, dataset_id='normal')
print(f"Staged normal operations: {df_staged_normal.shape}")
display(df_staged_normal.head(3))

Staged normal operations: (1209601, 129)


,dataset_id,time_utc,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,1_MV_004_STATUS,1_P_001_STATUS,1_P_002_STATUS,1_P_003_STATUS,1_P_004_STATUS,1_P_005_STATUS,1_P_006_STATUS,2_DPIT_001_PV,2_FIC_101_CO,2_FIC_101_PV,2_FIC_101_SP,2_FIC_201_CO,2_FIC_201_PV,2_FIC_201_SP,2_FIC_301_CO,2_FIC_301_PV,2_FIC_301_SP,2_FIC_401_CO,2_FIC_401_PV,2_FIC_401_SP,2_FIC_501_CO,2_FIC_501_PV,2_FIC_501_SP,2_FIC_601_CO,2_FIC_601_PV,2_FIC_601_SP,2_FIT_001_PV,2_FIT_002_PV,2_FIT_003_PV,2_FQ_101_PV,2_FQ_201_PV,2_FQ_301_PV,2_FQ_401_PV,2_FQ_501_PV,2_FQ_601_PV,2_LS_001_AL,2_LS_002_AL,2_LS_101_AH,2_LS_101_AL,2_LS_201_AH,2_LS_201_AL,2_LS_301_AH,2_LS_301_AL,2_LS_401_AH,2_LS_401_AL,2_LS_501_AH,2_LS_501_AL,2_LS_601_AH,2_LS_601_AL,2_LT_001_PV,2_LT_002_PV,2_MCV_007_CO,2_MCV_101_CO,2_MCV_201_CO,2_MCV_301_CO,2_MCV_401_CO,2_MCV_501_CO,2_MCV_601_CO,2_MV_001_STATUS,2_MV_002_STATUS,2_MV_003_STATUS,2_MV_004_STATUS,2_MV_005_STATUS,2_MV_006_STATUS,2_MV_009_STATUS,2_MV_101_STATUS,2_MV_201_STATUS,2_MV_301_STATUS,2_MV_401_STATUS,2_MV_501_STATUS,2_MV_601_STATUS,2_P_001_STATUS,2_P_002_STATUS,2_P_003_SPEED,2_P_003_STATUS,2_P_004_SPEED,2_P_004_STATUS,2_PIC_003_CO,2_PIC_003_PV,2_PIC_003_SP,2_PIT_001_PV,2_PIT_002_PV,2_PIT_003_PV,2_SV_101_STATUS,2_SV_201_STATUS,2_SV_301_STATUS,2_SV_401_STATUS,2_SV_501_STATUS,2_SV_601_STATUS,2A_AIT_001_PV,2A_AIT_002_PV,2A_AIT_003_PV,2A_AIT_004_PV,2B_AIT_001_PV,2B_AIT_002_PV,2B_AIT_003_PV,2B_AIT_004_PV,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LS_001_AL,3_LT_001_PV,3_MV_001_STATUS,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW
0,normal,2017-09-25 18:00:00+00:00,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,1,1,1,1,1,1,1,2,1,2464.88,9.08055,0.175914,0.13,10.6977,0.10365,0.11,6.81561,0.134973,0.1,9.51929,0.116865,0.15,6.57709,0.085249,0.1,6.04567,0.092886,0.09,0.644631,0.0,0.805636,0.183299,0.098509,0.136653,0.116025,0.086063,0.090545,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,70.4164,68.0508,0,6.03246,6.55246,9.50342,6.83231,10.7173,9.12922,1,1,2,2,2,2,2,1,1,1,1,1,1,NaN,NaN,24.8465,2,-0.006716,1,29.6518,1.81177,1,150.392,87.0202,1.81177,1,1,1,1,1,1,0.177734,8.850320e-41,8.37500,478.641,0.198242,8.97110,8.66633,472.527,0.0,0.0,1.18417,618.079,0.375448,0.001268,1,64.3425,1,1,1,1,1,1,1,67.9651,1,0.68
1,normal,2017-09-25 18:00:01+00:00,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,1,1,1,1,1,1,1,2,1,2464.88,9.08055,0.175914,0.13,10.6977,0.10365,0.11,6.81561,0.134973,0.1,9.51929,0.116865,0.15,6.57709,0.085249,0.1,6.04567,0.092886,0.09,0.644631,0.0,0.805636,0.183299,0.098509,0.136653,0.116025,0.086063,0.090545,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,70.4164,68.0508,0,6.03246,6.55246,9.50342,6.83231,10.7173,9.12922,1,1,2,2,2,2,2,1,1,1,1,1,1,NaN,NaN,24.8465,2,-0.006716,1,29.6518,1.81177,1,150.392,87.0202,1.81177,1,1,1,1,1,1,0.177734,1.394330e-01,8.42676,476.980,0.198242,8.89317,8.62920,472.415,0.0,0.0,1.18417,618.079,0.375448,0.001268,1,64.3425,1,1,1,1,1,1,1,67.9651,1,0.68
2,normal,2017-09-25 18:00:02+00:00,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,1,1,1,1,1,1,1,2,1,2464.88,9.08055,0.175914,0.13,10.6977,0.10365,0.11,6.81561,0.134973,0.1,9.51929,0.116865,0.15,6.57709,0.085249,0.1,6.04567,0.092886,0.09,0.644631,0.0,0.805636,0.183299,0.098509,0.136653,0.116025,0.086063,0.090545,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,70.4164,68.0508,0,6.03246,6.55246,9.50342,6.83231,10.7173,9.12922,1,1,2,2,2,2,2,1,1,1,1,1,1,NaN,NaN,24.8465,2,-0.006716,1,29.6518,1.81177,1,150.392,87.0202,1.81177,1,1,1,1,1,1,0.177734,1.394330e-01,8.42676,476.980,0.198242,8.89317,8.62920,472.415,0.0,0.0,1.18417,618.079,0.375448,0.001268,1,64.3425,1,1,1,1,1,1,1,67.9651,1,0.68


In [31]:
# Stage attack data
df_staged_attack = stage_wadi_data(df_raw_attack, dataset_id='attack')
print(f"\nStaged attack data: {df_staged_attack.shape}")
display(df_staged_attack.head(3))


Staged attack data: (172801, 129)


,dataset_id,time_utc,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,1_MV_004_STATUS,1_P_001_STATUS,1_P_002_STATUS,1_P_003_STATUS,1_P_004_STATUS,1_P_005_STATUS,1_P_006_STATUS,2_DPIT_001_PV,2_FIC_101_CO,2_FIC_101_PV,2_FIC_101_SP,2_FIC_201_CO,2_FIC_201_PV,2_FIC_201_SP,2_FIC_301_CO,2_FIC_301_PV,2_FIC_301_SP,2_FIC_401_CO,2_FIC_401_PV,2_FIC_401_SP,2_FIC_501_CO,2_FIC_501_PV,2_FIC_501_SP,2_FIC_601_CO,2_FIC_601_PV,2_FIC_601_SP,2_FIT_001_PV,2_FIT_002_PV,2_FIT_003_PV,2_FQ_101_PV,2_FQ_201_PV,2_FQ_301_PV,2_FQ_401_PV,2_FQ_501_PV,2_FQ_601_PV,2_LS_001_AL,2_LS_002_AL,2_LS_101_AH,2_LS_101_AL,2_LS_201_AH,2_LS_201_AL,2_LS_301_AH,2_LS_301_AL,2_LS_401_AH,2_LS_401_AL,2_LS_501_AH,2_LS_501_AL,2_LS_601_AH,2_LS_601_AL,2_LT_001_PV,2_LT_002_PV,2_MCV_007_CO,2_MCV_101_CO,2_MCV_201_CO,2_MCV_301_CO,2_MCV_401_CO,2_MCV_501_CO,2_MCV_601_CO,2_MV_001_STATUS,2_MV_002_STATUS,2_MV_003_STATUS,2_MV_004_STATUS,2_MV_005_STATUS,2_MV_006_STATUS,2_MV_009_STATUS,2_MV_101_STATUS,2_MV_201_STATUS,2_MV_301_STATUS,2_MV_401_STATUS,2_MV_501_STATUS,2_MV_601_STATUS,2_P_001_STATUS,2_P_002_STATUS,2_P_003_SPEED,2_P_003_STATUS,2_P_004_SPEED,2_P_004_STATUS,2_PIC_003_CO,2_PIC_003_PV,2_PIC_003_SP,2_PIT_001_PV,2_PIT_002_PV,2_PIT_003_PV,2_SV_101_STATUS,2_SV_201_STATUS,2_SV_301_STATUS,2_SV_401_STATUS,2_SV_501_STATUS,2_SV_601_STATUS,2A_AIT_001_PV,2A_AIT_002_PV,2A_AIT_003_PV,2A_AIT_004_PV,2B_AIT_001_PV,2B_AIT_002_PV,2B_AIT_003_PV,2B_AIT_004_PV,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LS_001_AL,3_LT_001_PV,3_MV_001_STATUS,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW
0,attack,2017-10-09 18:00:00+00:00,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,0,48.482,1,1,1,1,1,1,1,1,1,1,2538.7,35.6659,0.240392,0.2,68.5648,0.306246,0.1,100.0,0.04655,0.1,20.0701,0.175445,0.09,100.0,0.0488,0.11,100.0,0.051166,0.15,0.0,0.61969,0.0,0.239192,0.310242,0.04655,0.170598,0.0488,0.051166,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,69.8595,78.5906,0,0.0,0.0,20.0685,0.0,68.6983,35.6947,1,1,1,2,2,1,2,1,1,2,1,2,2,NaN,NaN,-0.016157,1,0.029574,1,50.0,0.025071,1.0,152.437,89.5817,0.025071,1,1,1,1,1,1,0.168945,7.805650e-41,8.6875,473.89,0.188477,8.96011,8.69146,475.488,0,8279.1,11.528,-65.9686,0.755703,0.626306,1,64.3306,1,1,1,1,1,1,1,62.6226,1,0.39
1,attack,2017-10-09 18:00:01+00:00,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,0,48.482,1,1,1,1,1,1,1,1,1,1,2538.7,35.6659,0.240392,0.2,68.5648,0.306246,0.1,100.0,0.04655,0.1,20.0701,0.175445,0.09,100.0,0.0488,0.11,100.0,0.051166,0.15,0.0,0.61969,0.0,0.239192,0.310242,0.04655,0.170598,0.0488,0.051166,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,69.8595,78.5906,0,0.0,0.0,20.0685,0.0,68.6983,35.6947,1,1,1,2,2,1,2,1,1,2,1,2,2,NaN,NaN,-0.016157,1,0.029574,1,50.0,0.025071,1.0,152.437,89.5817,0.025071,1,1,1,1,1,1,0.168945,7.805650e-41,8.6875,473.89,0.188477,8.96011,8.69146,475.488,0,8279.1,11.528,-65.9686,0.755703,0.626306,1,64.3306,1,1,1,1,1,1,1,62.6226,1,0.39
2,attack,2017-10-09 18:00:02+00:00,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,0,48.482,1,1,1,1,1,1,1,1,1,1,2538.7,35.6659,0.240392,0.2,68.5648,0.306246,0.1,100.0,0.04655,0.1,20.0701,0.175445,0.09,100.0,0.0488,0.11,100.0,0.051166,0.15,0.0,0.61969,0.0,0.239192,0.310242,0.04655,0.170598,0.0488,0.051166,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,69.8595,78.5906,0,0.0,0.0,20.0685,0.0,68.6983,35.6947,1,1,1,2,2,1,2,1,1,2,1,2,2,NaN,NaN,-0.016157,1,0.029574,1,50.0,0.025071,1.0,152.437,89.5817,0.025071,1,1,1,1,1,1,0.168945,7.805650e-41,8.6875,473.89,0.188477,8.96011,8.69146,475.488,0,8279.1,11.528,-65.9686,0.755703,0.626306,1,64.3306,1,1,1,1,1,1,1,62.6226,1,0.39


## 2.4 Write Staged Outputs

**How do we save it?**

In [32]:
# Write staged data

# Save normal ops
staged_normal_path = STAGED_DIR / f'wadi_staged_normal_{RUN_ID}.parquet'
df_staged_normal.to_parquet(staged_normal_path, index=False)
print(f"Wrote: {staged_normal_path}")

# Save attack data
staged_attack_path = STAGED_DIR / f'wadi_staged_attack_{RUN_ID}.parquet'
df_staged_attack.to_parquet(staged_attack_path, index=False)
print(f"Wrote: {staged_attack_path}")

# Create staged metadata
staged_metadata = {
    'run_id': RUN_ID,
    'staged_at_utc': utc_now_iso(),
    'datasets': {
        'normal': {
            'path': staged_normal_path.name,
            'rows': len(df_staged_normal),
            'columns': len(df_staged_normal.columns),
            'time_range': {
                'start': df_staged_normal['time_utc'].min().isoformat(),
                'end': df_staged_normal['time_utc'].max().isoformat()
            }
        },
        'attack': {
            'path': staged_attack_path.name,
            'rows': len(df_staged_attack),
            'columns': len(df_staged_attack.columns),
            'time_range': {
                'start': df_staged_attack['time_utc'].min().isoformat(),
                'end': df_staged_attack['time_utc'].max().isoformat()
            }
        }
    }
}

staged_meta_path = STAGED_DIR / f'staged_metadata_{RUN_ID}.json'
write_json(staged_meta_path, staged_metadata)
print(f"Wrote: {staged_meta_path}")

Wrote: work/wadi_A1/data/staged/wadi_staged_normal_20260207_142130_UTC.parquet
Wrote: work/wadi_A1/data/staged/wadi_staged_attack_20260207_142130_UTC.parquet
Wrote: work/wadi_A1/data/staged/staged_metadata_20260207_142130_UTC.json


---

# 3. Curate: Analysis-Ready Features

Add engineered features and data quality flags.

## 3.1 Create Curated DataFrame

In [33]:
# Create curated DataFrame

def curate_wadi_data(df_normal: pd.DataFrame, df_attack: pd.DataFrame) -> pd.DataFrame:

    # Add labels
    df_normal = df_normal.copy()
    df_attack = df_attack.copy()

    df_normal['label'] = 0
    df_attack['label'] = 1

    # Combine datasets
    df = pd.concat([df_normal, df_attack], ignore_index=True)

    # Sort by time
    df = df.sort_values('time_utc').reset_index(drop=True)

    # Add time featuers
    df['observation_day'] = df['time_utc'].dt.date
    df['observation_hour'] = df['time_utc'].dt.hour
    df['dayofweek'] = df['time_utc'].dt.dayofweek
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

    return df

print("Curate function ready")

Curate function ready


## 3.2 Test Curation Function

In [34]:
# Test Curation Function

df_curated = curate_wadi_data(df_staged_normal, df_staged_attack)

print(f"Curated dataset: {df_curated.shape}")
print(f"\nLabel distribution:")
print(df_curated['label'].value_counts().sort_index())
print(f"\nTime range:")
print(f"  Start: {df_curated['time_utc'].min()}")
print(f"  End: {df_curated['time_utc'].max()}")
print(f"\nColumn count: {len(df_curated.columns)}")

display(df_curated.head(3))


Curated dataset: (1382402, 134)

Label distribution:
label
0    1209601
1     172801
Name: count, dtype: int64

Time range:
  Start: 2017-09-25 18:00:00+00:00
  End: 2017-10-11 18:00:00+00:00

Column count: 134


,dataset_id,time_utc,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,1_MV_004_STATUS,1_P_001_STATUS,1_P_002_STATUS,1_P_003_STATUS,1_P_004_STATUS,1_P_005_STATUS,1_P_006_STATUS,2_DPIT_001_PV,2_FIC_101_CO,2_FIC_101_PV,2_FIC_101_SP,2_FIC_201_CO,2_FIC_201_PV,2_FIC_201_SP,2_FIC_301_CO,2_FIC_301_PV,2_FIC_301_SP,2_FIC_401_CO,2_FIC_401_PV,2_FIC_401_SP,2_FIC_501_CO,2_FIC_501_PV,2_FIC_501_SP,2_FIC_601_CO,2_FIC_601_PV,2_FIC_601_SP,2_FIT_001_PV,2_FIT_002_PV,2_FIT_003_PV,2_FQ_101_PV,2_FQ_201_PV,2_FQ_301_PV,2_FQ_401_PV,2_FQ_501_PV,2_FQ_601_PV,2_LS_001_AL,2_LS_002_AL,2_LS_101_AH,2_LS_101_AL,2_LS_201_AH,2_LS_201_AL,2_LS_301_AH,2_LS_301_AL,2_LS_401_AH,2_LS_401_AL,2_LS_501_AH,2_LS_501_AL,2_LS_601_AH,2_LS_601_AL,2_LT_001_PV,2_LT_002_PV,2_MCV_007_CO,2_MCV_101_CO,2_MCV_201_CO,2_MCV_301_CO,2_MCV_401_CO,2_MCV_501_CO,2_MCV_601_CO,2_MV_001_STATUS,2_MV_002_STATUS,2_MV_003_STATUS,2_MV_004_STATUS,2_MV_005_STATUS,2_MV_006_STATUS,2_MV_009_STATUS,2_MV_101_STATUS,2_MV_201_STATUS,2_MV_301_STATUS,2_MV_401_STATUS,2_MV_501_STATUS,2_MV_601_STATUS,2_P_001_STATUS,2_P_002_STATUS,2_P_003_SPEED,2_P_003_STATUS,2_P_004_SPEED,2_P_004_STATUS,2_PIC_003_CO,2_PIC_003_PV,2_PIC_003_SP,2_PIT_001_PV,2_PIT_002_PV,2_PIT_003_PV,2_SV_101_STATUS,2_SV_201_STATUS,2_SV_301_STATUS,2_SV_401_STATUS,2_SV_501_STATUS,2_SV_601_STATUS,2A_AIT_001_PV,2A_AIT_002_PV,2A_AIT_003_PV,2A_AIT_004_PV,2B_AIT_001_PV,2B_AIT_002_PV,2B_AIT_003_PV,2B_AIT_004_PV,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LS_001_AL,3_LT_001_PV,3_MV_001_STATUS,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW,label,observation_day,observation_hour,dayofweek,is_weekend
0,normal,2017-09-25 18:00:00+00:00,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,1,1,1,1,1,1,1,2,1,2464.88,9.08055,0.175914,0.13,10.6977,0.10365,0.11,6.81561,0.134973,0.1,9.51929,0.116865,0.15,6.57709,0.085249,0.1,6.04567,0.092886,0.09,0.644631,0.0,0.805636,0.183299,0.098509,0.136653,0.116025,0.086063,0.090545,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,70.4164,68.0508,0,6.03246,6.55246,9.50342,6.83231,10.7173,9.12922,1,1,2,2,2,2,2,1,1,1,1,1,1,NaN,NaN,24.8465,2,-0.006716,1,29.6518,1.81177,1.0,150.392,87.0202,1.81177,1,1,1,1,1,1,0.177734,8.850320e-41,8.37500,478.641,0.198242,8.97110,8.66633,472.527,0.0,0.0,1.18417,618.079,0.375448,0.001268,1,64.3425,1,1,1,1,1,1,1,67.9651,1,0.68,0,2017-09-25,18,0,0
1,normal,2017-09-25 18:00:01+00:00,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,1,1,1,1,1,1,1,2,1,2464.88,9.08055,0.175914,0.13,10.6977,0.10365,0.11,6.81561,0.134973,0.1,9.51929,0.116865,0.15,6.57709,0.085249,0.1,6.04567,0.092886,0.09,0.644631,0.0,0.805636,0.183299,0.098509,0.136653,0.116025,0.086063,0.090545,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,70.4164,68.0508,0,6.03246,6.55246,9.50342,6.83231,10.7173,9.12922,1,1,2,2,2,2,2,1,1,1,1,1,1,NaN,NaN,24.8465,2,-0.006716,1,29.6518,1.81177,1.0,150.392,87.0202,1.81177,1,1,1,1,1,1,0.177734,1.394330e-01,8.42676,476.980,0.198242,8.89317,8.62920,472.415,0.0,0.0,1.18417,618.079,0.375448,0.001268,1,64.3425,1,1,1,1,1,1,1,67.9651,1,0.68,0,2017-09-25,18,0,0
2,normal,2017-09-25 18:00:02+00:00,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,1,1,1,1,1,1,1,2,1,2464.88,9.08055,0.175914,0.13,10.6977,0.10365,0.11,6.81561,0.134973,0.1,9.51929,0.116865,0.15,6.57709,0.085249,0.1,6.04567,0.092886,0.09,0.644631,0.0,0.805636,0.183299,0.098509,0.136653,0.116025,0.086063,0.090545,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,70.4164,68.0508,0,6.03246,6.55246,9.50342,6.83231,10.7173,9.12922,1,1,2,2,2,2,2,1,1,1,1,1,1,NaN,NaN,24.8465,2,-0.006716,1,29.6518,1.81177,1.0,150.392,87.0202,1.81177,1,1,1,1,1,1,0.177734,1.394330e-01,8.42676,476.980,0.198242,8.89317,8.62920,472.415,0.0,0.0,1.18417,618.079,0.375448,0.001268,1,64.3425,1,1,1,1,1,1,1,67.9651,1,0.68,0,2017-09-25,18,0,0


## 3.3 Choose Curated Columns and Write

In [35]:
# Write curated data

# Save curated dataset
curated_path = WH_DIR / f'wadi_curated_{RUN_ID}.parquet'
df_curated.to_parquet(curated_path, index=False)
print(f"Wrote: {curated_path}")

# Create curated metadata
curated_metadata = {
    'run_id': RUN_ID,
    'curated_at_utc': utc_now_iso(),
    'source_files': {
        'normal': f'wadi_staged_normal_{RUN_ID}.parquet',
        'attack': f'wadi_staged_attack_{RUN_ID}.parquet'
    },
    'curated_file': curated_path.name,
    'rows': len(df_curated),
    'columns': len(df_curated.columns),
    'label_distribution': df_curated['label'].value_counts().sort_index().to_dict(),
    'time_range': {
        'start': df_curated['time_utc'].min().isoformat(),
        'end': df_curated['time_utc'].max().isoformat(),
        'span_days': (df_curated['time_utc'].max() - df_curated['time_utc'].min()).days
    },
    'features_added': ['label', 'observation_day', 'observation_hour', 'dayofweek', 'is_weekend'],
    'notes': [
        'Combined normal operations and attack datasets',
        'Label: 0=normal, 1=attack',
        'Sensor failures not yet added (future step)',
        'Data sorted by time_utc'
    ]
}

curated_meta_path = WH_DIR / f'curated_metadata_{RUN_ID}.json'
write_json(curated_meta_path, curated_metadata)
print(f"Wrote: {curated_meta_path}")

Wrote: work/wadi_A1/data/warehouse/wadi_curated_20260207_142130_UTC.parquet
Wrote: work/wadi_A1/data/warehouse/curated_metadata_20260207_142130_UTC.json


---

# 4. Validate: Contracts + Anomalies + Canaries

Define validation rules and check data quality

## 4.1 Define Contracts

**Define Rules**

In [36]:
# Define contracts

def create_validation_contracts(df: pd.DataFrame) -> dict:
    contracts = {
        'required_columns': ['dataset_id', 'time_utc', 'label', 'observation_day'],
        'valid_labels': [0, 1],
        'time_must_be_sorted': True,
        'allow_duplicate_timestamps': False,
        'critical_fields_no_nulls': ['time_utc', 'label', 'dataset_id']
    }

    return contracts

print("Validation Contracts ready")

Validation Contracts ready


## 4.2 Implement Validation Checks

**Implement Checker**

In [42]:
# Implement validation checks

def validate_contracts(df: pd.DataFrame, contracts: dict) -> dict:
    results = {}

    # Required columns
    required = contracts['required_columns']
    missing = [c for c in required if c not in df.columns]
    results['required_columns'] = {
        'expected': required,
        'missing': missing,
        'passed': len(missing) == 0
    }

    # Label values
    valid_labels = set(contracts['valid_labels'])
    actual_labels = set(df['label'].unique())
    invalid = actual_labels - valid_labels
    results['label_values'] = {
        'expected': contracts['valid_labels'],
        'actual': list(actual_labels),
        'invalid': list(invalid),
        'passed': len(invalid) == 0
    }

    # Time ordering
    if contracts['time_must_be_sorted']:
        is_sorted = df['time_utc'].is_monotonic_increasing
        results['time_ordering'] = {
            'is_sorted': is_sorted,
            'passed': is_sorted
        }

    # Duplicate timestamps
    if not contracts['allow_duplicate_timestamps']:
        duplicates = df.groupby(['dataset_id', 'time_utc']).size()
        dupe_count = int((duplicates > 1).sum())
        results['duplicate_timestamps'] = {
            'count': dupe_count,
            'passed': dupe_count == 0
        }

    # Critical fields null
    null_counts = {}
    for col in contracts['critical_fields_no_nulls']:
        null_counts[col] = int(df[col].isna().sum())

    results['null_critical_fields'] = {
        'null_counts': null_counts,
        'passed': all(count == 0 for count in null_counts.values())
    }

    # Overall
    results['all_passed'] = all(r['passed'] for r in results.values() if 'passed' in r)

    return results

print("Validation function ready")

    
# Return validation report with:
# - passed (True/False)
# - failures (list)
# - warnings (list)

Validation function ready


## 4.3 Run Validation

**Run Checker**

In [43]:
# Validation

contracts = create_validation_contracts(df_curated)
validation_report = validate_contracts(df_curated, contracts)

print("VALIDATION Report")
for check_name, result in validation_report.items():
    if check_name == 'all_passed':
        continue

    status = "PASS" if result.get('passed', False) else "FAIL"
    print(f"\n{check_name}: {status}")

    for key, value in result.items():
        if key != 'passed':
            print(f" {key}: {value}")

print(f"Overall: {'ALL PASSED' if validation_report['all_passed'] else 'FAILED'}")


VALIDATION Report

required_columns: PASS
 expected: ['dataset_id', 'time_utc', 'label', 'observation_day']
 missing: []

label_values: PASS
 expected: [0, 1]
 actual: [np.int64(0), np.int64(1)]
 invalid: []

time_ordering: PASS
 is_sorted: True

duplicate_timestamps: PASS
 count: 0

null_critical_fields: PASS
 null_counts: {'time_utc': 0, 'label': 0, 'dataset_id': 0}
Overall: ALL PASSED


## 4.4 Anomaly Flags and Investigation

**Define Anomalies**

In [44]:
# Define anomaly detection

def create_anomaly_summary(df: pd.DataFrame) -> dict:
    """
    Detect anomalies in sensor data.
    
    Returns:
        Dictionary of anomaly findings
    """
    summary = {}
    
    # Get sensor columns (exclude metadata and time features)
    non_sensor_cols = ['dataset_id', 'time_utc', 'label', 'observation_day', 
                       'observation_hour', 'dayofweek', 'is_weekend']
    sensor_cols = [c for c in df.columns if c not in non_sensor_cols]
    
    # Missingness by column
    missingness = {}
    for col in sensor_cols:
        miss_pct = (df[col].isna().sum() / len(df)) * 100
        if miss_pct > 0:
            missingness[col] = round(miss_pct, 2)
    
    summary['high_missingness'] = {
        col: pct for col, pct in sorted(missingness.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    summary['total_columns_with_missing'] = len(missingness)
    
    # Constant columns (no variation)
    constant_cols = []
    for col in sensor_cols:
        if df[col].nunique() == 1:
            constant_cols.append(col)
    
    summary['constant_columns'] = constant_cols
    summary['constant_count'] = len(constant_cols)
    
    # Suspicious zero-only columns
    zero_only = []
    for col in sensor_cols:
        non_null = df[col].dropna()
        if len(non_null) > 0 and (non_null == 0).all():
            zero_only.append(col)
    
    summary['zero_only_columns'] = zero_only
    summary['zero_only_count'] = len(zero_only)
    
    return summary

print("Anomaly detection function ready")

Anomaly detection function ready


## 4.5 Run Anomaly Analysis

In [45]:
# Run anomaly analysis

anomaly_summary = create_anomaly_summary(df_curated)

print("ANOMALY ANALYSIS")

print(f"\nTotal columns with missing data: {anomaly_summary['total_columns_with_missing']}")
print(f"\nTop 10 columns by missingness (%):")
for col, pct in anomaly_summary['high_missingness'].items():
    print(f"  {col}: {pct}%")

print(f"\nConstant columns (no variation): {anomaly_summary['constant_count']}")
if anomaly_summary['constant_columns']:
    for col in anomaly_summary['constant_columns'][:5]:
        print(f"  {col}")
    if len(anomaly_summary['constant_columns']) > 5:
        print(f"  ... and {len(anomaly_summary['constant_columns']) - 5} more")

print(f"\nZero-only columns: {anomaly_summary['zero_only_count']}")
if anomaly_summary['zero_only_columns']:
    for col in anomaly_summary['zero_only_columns'][:5]:
        print(f"  {col}")
    if len(anomaly_summary['zero_only_columns']) > 5:
        print(f"  ... and {len(anomaly_summary['zero_only_columns']) - 5} more")

print("=" * 60)

# Create investigation table
print("\nANOMALY INVESTIGATION TABLE")
investigation_data = []

# Add high missingness issues
for col, pct in list(anomaly_summary['high_missingness'].items())[:5]:
    investigation_data.append({
        'issue': 'high_missingness',
        'column': col,
        'value': f"{pct}%",
        'action': 'Review if sensor was offline or feature should be excluded'
    })

# Add constant columns
for col in anomaly_summary['constant_columns'][:5]:
    investigation_data.append({
        'issue': 'constant_value',
        'column': col,
        'value': 'no variation',
        'action': 'Consider removing - no predictive value'
    })

# Add zero-only columns
for col in anomaly_summary['zero_only_columns'][:5]:
    investigation_data.append({
        'issue': 'zero_only',
        'column': col,
        'value': 'all zeros',
        'action': 'Check if sensor malfunctioned or always inactive'
    })

if investigation_data:
    df_investigation = pd.DataFrame(investigation_data)
    display(df_investigation)
else:
    print("No anomalies detected")

ANOMALY ANALYSIS

Total columns with missing data: 9

Top 10 columns by missingness (%):
  2_LS_001_AL: 100.0%
  2_LS_002_AL: 100.0%
  2_P_001_STATUS: 100.0%
  2_P_002_STATUS: 100.0%
  1_AIT_002_PV: 0.0%
  1_AIT_004_PV: 0.0%
  2B_AIT_004_PV: 0.0%
  3_AIT_002_PV: 0.0%
  3_AIT_004_PV: 0.0%

Constant columns (no variation): 25
  1_LS_001_AL
  1_LS_002_AL
  1_P_002_STATUS
  1_P_004_STATUS
  2_MV_001_STATUS
  ... and 20 more

Zero-only columns: 2
  1_LS_001_AL
  1_LS_002_AL

ANOMALY INVESTIGATION TABLE


,issue,column,value,action
0,high_missingness,2_LS_001_AL,100.0%,Review if sensor was offline or feature should...
1,high_missingness,2_LS_002_AL,100.0%,Review if sensor was offline or feature should...
2,high_missingness,2_P_001_STATUS,100.0%,Review if sensor was offline or feature should...
3,high_missingness,2_P_002_STATUS,100.0%,Review if sensor was offline or feature should...
4,high_missingness,1_AIT_002_PV,0.0%,Review if sensor was offline or feature should...
5,constant_value,1_LS_001_AL,no variation,Consider removing - no predictive value
6,constant_value,1_LS_002_AL,no variation,Consider removing - no predictive value
7,constant_value,1_P_002_STATUS,no variation,Consider removing - no predictive value
8,constant_value,1_P_004_STATUS,no variation,Consider removing - no predictive value
9,constant_value,2_MV_001_STATUS,no variation,Consider removing - no predictive value


## 4.6 Canary Checks

**Define Canaries**

In [53]:
# Define canary checks

def create_canary_summary(df: pd.DataFrame) -> dict:
    summary = {}
    
    # Observations per day
    obs_per_day = df.groupby('observation_day').size()
    summary['observations_per_day'] = {
        'min': int(obs_per_day.min()),
        'median': int(obs_per_day.median()),
        'max': int(obs_per_day.max()),
        'mean': round(obs_per_day.mean(), 2)
    }
    
    # Days with drops (< 50% of median)
    median_obs = obs_per_day.median()
    threshold = median_obs * 0.5
    drop_days = obs_per_day[obs_per_day < threshold]
    summary['drop_days'] = {
        'count': len(drop_days),
        'threshold': int(threshold),
        'days': {str(k): v for k, v in drop_days.to_dict().items()} if len(drop_days) > 0 else {}
    }
    
    # Overall missingness by column
    non_sensor_cols = ['dataset_id', 'time_utc', 'label', 'observation_day', 
                       'observation_hour', 'dayofweek', 'is_weekend']
    sensor_cols = [c for c in df.columns if c not in non_sensor_cols]
    
    overall_miss = {}
    for col in sensor_cols:
        miss_pct = (df[col].isna().sum() / len(df)) * 100
        overall_miss[col] = round(miss_pct, 2)
    
    summary['overall_missingness'] = {
        'mean': round(sum(overall_miss.values()) / len(overall_miss), 2),
        'max': max(overall_miss.values()),
        'columns_over_10pct': sum(1 for pct in overall_miss.values() if pct > 10)
    }
    
    # Worst day missingness
    missingness_by_day = df.groupby('observation_day').apply(
        lambda g: (g[sensor_cols].isna().sum().sum() / (len(g) * len(sensor_cols))) * 100,
        include_groups=False
    )
    worst_day = missingness_by_day.idxmax()
    summary['worst_day_missingness'] = {
        'date': str(worst_day),
        'missingness_pct': round(missingness_by_day.max(), 2)
    }
    
    # Days with high missingness (> 30%)
    high_miss_days = missingness_by_day[missingness_by_day > 30]
    summary['high_missingness_days'] = {
        'count': len(high_miss_days),
        'days': high_miss_days.to_dict() if len(high_miss_days) > 0 else {}
    }
    
    return summary

print("Canary function ready")

Canary function ready


## 4.7 Run Canary Analysis

In [54]:
# Run canary analysis

canary_summary = create_canary_summary(df_curated)

print("\nObservations per day:")
for key, value in canary_summary['observations_per_day'].items():
    print(f"  {key}: {value}")

print(f"\nDrop days (< 50% of median): {canary_summary['drop_days']['count']}")
if canary_summary['drop_days']['days']:
    print(f"  Threshold: {canary_summary['drop_days']['threshold']} observations")
    for day, count in list(canary_summary['drop_days']['days'].items())[:5]:
        print(f"  {day}: {count} observations")

print("\nOverall missingness:")
for key, value in canary_summary['overall_missingness'].items():
    print(f"  {key}: {value}")

print(f"\nWorst day missingness:")
print(f"  Date: {canary_summary['worst_day_missingness']['date']}")
print(f"  Missingness: {canary_summary['worst_day_missingness']['missingness_pct']}%")

print(f"\nHigh missingness days (>30%): {canary_summary['high_missingness_days']['count']}")
if canary_summary['high_missingness_days']['days']:
    for day, pct in list(canary_summary['high_missingness_days']['days'].items())[:5]:
        print(f"  {day}: {pct:.2f}%")



Observations per day:
  min: 21600
  median: 86400
  max: 86401
  mean: 81317.76

Drop days (< 50% of median): 1
  Threshold: 43200 observations
  2017-09-25: 21600 observations

Overall missingness:
  mean: 3.15
  max: 100.0
  columns_over_10pct: 4

Worst day missingness:
  Date: 2017-09-26
  Missingness: 3.15%

High missingness days (>30%): 0


---

# 5. Leakage Audit (Conceptual)

Document potential temporal leakage risks.

## 5.1 Write Leakage Checklist

In [55]:
# Leakage checklist

leakage_checklist = [
    "Data sorted by time - can split chronologically for train/test",
    "No future information in features - only time_utc and derived time features added",
    "If building rolling/lag features, ensure computed using only past data",
    "If standardizing/normalizing sensors, compute stats on TRAIN only, then apply to test",
    "When adding sensor failures, ensure failure timing doesn't leak attack labels",
    "observation_day available for proper temporal splits",
    "Attack labels come from metadata timestamps, not from looking at sensor values",
    "dataset_id could leak train/test split if not handled carefully - consider removing before modeling",
    "Future consideration: ensure any imputation only uses past observations",
    "Future consideration: rolling window features should use strict past-only data"
]

for i, item in enumerate(leakage_checklist, 1):
    print(f"{i}. {item}")

1. Data sorted by time - can split chronologically for train/test
2. No future information in features - only time_utc and derived time features added
3. If building rolling/lag features, ensure computed using only past data
4. If standardizing/normalizing sensors, compute stats on TRAIN only, then apply to test
5. When adding sensor failures, ensure failure timing doesn't leak attack labels
6. observation_day available for proper temporal splits
7. Attack labels come from metadata timestamps, not from looking at sensor values
8. dataset_id could leak train/test split if not handled carefully - consider removing before modeling
9. Future consideration: ensure any imputation only uses past observations
10. Future consideration: rolling window features should use strict past-only data


---

# 6. Write Final Artifacts

Consolidate validation results and create run log

## 6.1 Write Validation Report

In [56]:
# Write validation report

def write_validation_report(validation_report: dict, anomaly_summary: dict, 
                           canary_summary: dict, leakage_checklist: list) -> None:

    full_report = {
        'generated_at_utc': utc_now_iso(),
        'run_id': RUN_ID,
        'validation': validation_report,
        'anomalies': anomaly_summary,
        'canaries': canary_summary,
        'leakage_audit': leakage_checklist
    }
    
    report_path = REF_DIR / 'validation_report.json'
    write_json(report_path, full_report)
    
    print(f"Validation report written to: {report_path}")
    
    return report_path

# Write the report
validation_report_path = write_validation_report(
    validation_report, 
    anomaly_summary, 
    canary_summary, 
    leakage_checklist
)

Validation report written to: work/wadi_A1/data/reference/validation_report.json


## 6.2 Create Run Log

In [57]:
# Create run log

def create_run_log() -> None:
    """
    Document pipeline execution details for reproducibility.
    """
    run_log = {
        'run_id': RUN_ID,
        'generated_at_utc': utc_now_iso(),
        'inputs': {
            'raw_files': {
                'normal_ops': 'WADI_14days.csv',
                'attack_data': 'WADI_attackdata.csv',
                'attack_metadata': 'attack_description.xlsx'
            },
            'download_date': '2026-02-06',
            'source': DATA_SOURCE,
            'dataset_name': DATASET_NAME
        },
        'outputs': {
            'staged': {
                'normal': f'wadi_staged_normal_{RUN_ID}.parquet',
                'attack': f'wadi_staged_attack_{RUN_ID}.parquet',
                'metadata': f'staged_metadata_{RUN_ID}.json'
            },
            'curated': {
                'data': f'wadi_curated_{RUN_ID}.parquet',
                'metadata': f'curated_metadata_{RUN_ID}.json'
            },
            'validation': 'validation_report.json'
        },
        'row_definition': 'Each row represents one sensor observation at a specific UTC timestamp',
        'transformations': [
            'Cleaned column names (removed Windows path prefix)',
            'Parsed Date + Time into timezone-aware time_utc',
            'Combined normal and attack datasets',
            'Added labels: 0=normal, 1=attack',
            'Added time features: observation_day, observation_hour, dayofweek, is_weekend',
            'Sorted by time_utc'
        ],
        'data_summary': {
            'total_rows': len(df_curated),
            'total_columns': len(df_curated.columns),
            'label_distribution': df_curated['label'].value_counts().to_dict(),
            'time_span_days': (df_curated['time_utc'].max() - df_curated['time_utc'].min()).days
        },
        'notes': [
            'Sensor failures not yet added - will be separate preprocessing step',
            'Attack labels applied to entire attack dataset',
            'Future work: use attack_description.xlsx to label specific attack time windows'
        ]
    }
    
    run_log_path = RUN_DIR / f'{RUN_ID}.json'
    write_json(run_log_path, run_log)
    
    print(f"Run log written to: {run_log_path}")
    return run_log_path

# Create the run log
run_log_path = create_run_log()

Run log written to: work/wadi_A1/data/reference/pipeline_runs/20260207_142130_UTC.json


---

# 7. Self-Check and Reflection

Document insights and potential issues

## 7.1 Pipeline Reflection

In [58]:
# Pipeline reflection

reflection = [
    "Row definition: Each row represents one sensor observation from the WaDi water distribution testbed at a specific UTC timestamp.",
    
    "Required columns: dataset_id (source), time_utc (timestamp), label (0=normal, 1=attack), and 127 sensor columns from the industrial control system.",
    
    "Data cleaning: Removed Windows path prefix from column names (\\\\WIN-25J4RO10SBF\\LOG_DATA\\...), parsed separate Date/Time columns into timezone-aware UTC timestamps, combined normal operations (1.2M rows) with attack scenarios (173K rows).",
    
    "Biggest anomaly found: Several sensors show 100% missingness across the entire dataset, suggesting those sensors were never operational or data was not recorded. Zero-only and constant-value columns indicate some sensors may not be relevant for analysis.",
    
    "Likely breakage scenario: If new data arrives with different column names (sensor naming convention changes) or missing Date/Time columns, the staging function will fail. The pipeline assumes specific CSV structure (metadata rows for normal ops, direct headers for attack data).",
    
    "Temporal leakage risk: Main risk is if train/test split isn't done chronologically - dataset_id could leak information. Any future feature engineering (rolling stats, normalization) must use only past data. Sensor failure injection must not use information from attack labels.",
    
    "Data quality insight: Attack dataset is only ~14% of total data (173K vs 1.2M rows). This class imbalance will need to be addressed during model training. Time span is limited (16 days total), so model may not generalize to different operational conditions.",
    
    "Pipeline reproducibility: All transformations documented in run log. Raw data preserved in data/raw/, staged data in data/staged/, curated in data/warehouse/. RUN_ID timestamps allow tracking which outputs came from which execution. Validation report captures data quality state at pipeline execution time."
]

for item in reflection:
    if ':' in item:
        label, content = item.split(':', 1)
        print(f"{label}:")
        print(f"  {content.strip()}")
        print()
    else:
        print(item)
        print()



Row definition:
  Each row represents one sensor observation from the WaDi water distribution testbed at a specific UTC timestamp.

Required columns:
  dataset_id (source), time_utc (timestamp), label (0=normal, 1=attack), and 127 sensor columns from the industrial control system.

Data cleaning:
  Removed Windows path prefix from column names (\\WIN-25J4RO10SBF\LOG_DATA\...), parsed separate Date/Time columns into timezone-aware UTC timestamps, combined normal operations (1.2M rows) with attack scenarios (173K rows).

Biggest anomaly found:
  Several sensors show 100% missingness across the entire dataset, suggesting those sensors were never operational or data was not recorded. Zero-only and constant-value columns indicate some sensors may not be relevant for analysis.

Likely breakage scenario:
  If new data arrives with different column names (sensor naming convention changes) or missing Date/Time columns, the staging function will fail. The pipeline assumes specific CSV structure 

---

# 8. Final Summary

## Outputs Created:

## Outputs Created:

- `data/raw/` — raw data snapshot + metadata
- `data/staged/` — parsed, typed DataFrame
- `data/warehouse/` — analysis-ready curated data
- `data/reference/validation_report.json` — full validation results
- `data/reference/pipeline_runs/{run_id}.json` — execution log

## Next Steps:

1. Use curated data for analysis or modeling
2. Review validation report for data quality issues
3. Check leakage checklist before ML model development
4. Re-run pipeline with new data using same structure